In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from TinyImageNet import TinyImageNet
from tensorboardX import SummaryWriter
from torch.autograd import Variable


In [ ]:
from utils import show_images_horizontally
from NaiveResNet import NaiveResNet

In [ ]:
%reload_ext autoreload
%autoreload 2

## 步驟
- 讀取資料集
- 簡單 EDA
    - facets
- 定義目標 / loss function
- 定義模型
- 訓練模型
- 測試模型
- 視覺化 kernels / parameters

## 前處理資料
- 讀取資料
- 轉換（灰階處理、Augmentation、Crop）

注意在 validation 時我們不需要做 augmentation

In [ ]:
root = './dataset'

In [ ]:
# The output of torchvision datasets are PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1].
# normalize 在現在有 batch-normalization 的情況下其實非必要
normalize = transforms.Normalize((.5, .5, .5), (.5, .5, .5))

augmentation = transforms.RandomApply([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(64)], p=.8)

training_transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert("RGB")),
    augmentation,
    transforms.ToTensor(),
    normalize])

valid_transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert("RGB")),
    transforms.ToTensor(),
    normalize])

將圖片全讀到記憶體，最小化硬碟 overhead

In [ ]:
in_memory = False

In [ ]:
%%time
training_set = TinyImageNet(root, 'train', transform=training_transform, in_memory=in_memory)
valid_set = TinyImageNet(root, 'val', transform=valid_transform, in_memory=in_memory)

In [ ]:
training_set

## 顯示處理後圖片
主要是顯示經過 data augmentation 的圖片。為了讓模型更 robust，我們隨機進行水平翻轉、剪裁以及旋轉的處理。在這邊顯示的圖有進行反正規化（un-normalization）。

In [ ]:
tmpiter = iter(DataLoader(training_set, batch_size=10, shuffle=True))
for _ in range(5):
    images, labels = tmpiter.next()
    show_images_horizontally(images, un_normalize=True)

## 定義 loss function

In [ ]:
ce_loss = torch.nn.CrossEntropyLoss()

## 建立模型

In [ ]:
resnet = NaiveResNet(num_classes=200)
device = torch.device("cuda")
resnet = resnet.to(device)

## 將模型圖寫到 Tensorboard 以供確認

In [ ]:
from tensorboardX import SummaryWriter
sw = SummaryWriter(log_dir='./runs', comment='NaiveResNet')
dummy_input = Variable(torch.rand(16, 3, 64, 64)).to(device)
sw.add_graph(resnet, (dummy_input, ))

In [ ]:
# out = resnet.forward(dummy_input)
# out.size()

## 定義 Optimizer, Scheduler

In [ ]:
# optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 40)

## 訓練模型

In [ ]:
max_epochs = 120

In [ ]:
trainloader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=4)
validloader = DataLoader(valid_set, batch_size=64, num_workers=6)

In [ ]:
%%time
import time
assert torch.cuda.is_available()
try:
    for epoch in range(max_epochs):
        start = time.time()
        lr_scheduler.step()
        epoch_loss = 0.0
        resnet.train()
        for idx, (data, target) in enumerate(trainloader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = resnet(data)
            batch_loss = ce_loss(output, target)
            batch_loss.backward()
            optimizer.step()
            epoch_loss += batch_loss.item()
        
            if idx % 10 == 0:
                print('{:.1f}% of epoch'.format(idx / float(len(trainloader)) * 100), end='\r')
            
            
        # evaluate on validation set
        num_hits = 0
        num_instances = len(valid_set)
        
        with torch.no_grad():
            resnet.eval()
            for idx, (data, target) in enumerate(validloader):
                data, target = data.to(device), target.to(device)
                output = resnet(data)
                _, pred = torch.max(output, 1) # output.topk(1) *1 = top1

                num_hits += (pred == target).sum().item()
#                 print('{:.1f}% of validation'.format(idx / float(len(validloader)) * 100), end='\r')

        valid_acc = num_hits / num_instances * 100
        print(f' Validation acc: {valid_acc}%')
        sw.add_scalar('Validation Accuracy(%)', valid_acc, epoch)
            
        epoch_loss /= float(len(trainloader))
#         print("Time used in one epoch: {:.1f}".format(time.time() - start))
        
        # save model
        torch.save(resnet.state_dict(), 'models/weight.pth')
        
        # record loss
        sw.add_scalar('Running Loss', epoch_loss, epoch)
        
        
except KeyboardInterrupt:
    print("Interrupted. Releasing resources...")
    
finally:
    # this is only required for old GPU
    torch.cuda.empty_cache()

In [ ]:
!cd models/;ls -alth

## Load model

In [ ]:
sd = torch.load('models/weight.pth', map_location=lambda storage, location: storage)

In [ ]:
resnet.load_state_dict(sd)

In [ ]:
resnet = resnet.to(device)

## Todo
- Tensorboard
- save model by best metrics
- 多點augmentation
- 要不要加learning rate schduler
    - https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate